<a href="https://colab.research.google.com/github/TMUITLab/EAFR/blob/master/EA7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd '/content'
!git clone https://github.com/1049451037/GCN-Align
%cd '/content'
!git clone https://github.com/MaoXinn/RREA

In [4]:
import warnings
warnings.filterwarnings('ignore')


from importlib.machinery import SourceFileLoader
 
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()
utils = SourceFileLoader("utils", "/content/RREA/CIKM/utils.py").load_module()
CSLS = SourceFileLoader("CSLS", "/content/RREA/CIKM/CSLS.py").load_module()
import tensorflow as tf
import os
import random
import keras
from tqdm import *
import numpy as np
from utils import *
from CSLS import *
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from layer import NR_GraphAttention

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [5]:
lang = 'zh'
radj,train_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features = load_data('/content/GCN-Align/data/%s_en/'%lang,train_ratio=0.30)
train_pair_main=train_pair
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3024


In [7]:
entity1, rel1, triples1 = load_triples('/content/GCN-Align/data/%s_en/'%lang + 'triples_1')
num_entity_1 = len(entity1)
num_rel_1 = len(rel1)

In [31]:
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()

from layer import NR_GraphAttention,N3,AvgClass
tf.keras.backend.clear_session()

node_size = adj_features.shape[0]
rel_size = rel_features.shape[1]
triple_size = len(adj_matrix)
batch_size = node_size


class TokenEmbedding(keras.layers.Embedding):
    """Embedding layer with weights returned."""

    def compute_output_shape(self, input_shape):
        return self.input_dim, self.output_dim

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs):
        return self.embeddings
    
def get_sim_mat():
    inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,rest_set_1,rest_set_2]
    inputs = [np.expand_dims(item,axis=0) for item in inputs]
    return get_emb.predict_on_batch(inputs)

def get_losses_values():
    inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,train_set]
    inputs = [np.expand_dims(item,axis=0) for item in inputs]
    return get_losses.predict_on_batch(inputs )

def test(wrank = None):
    vec = get_embedding()
    return  get_hits(vec,dev_pair,wrank=wrank)

def CSLS_test(thread_number = 16, csls=10,accurate = True):
    vec = get_embedding()
    Lvec = np.array([vec[e1] for e1, e2 in dev_pair])
    Rvec = np.array([vec[e2] for e1, e2 in dev_pair])
    Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)
    eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], thread_number, csls=csls, accurate=accurate)
    return None

def get_train_set(batch_size = batch_size):
    negative_ratio =  batch_size // len(train_pair) + 1
    train_set = np.reshape(np.repeat(np.expand_dims(train_pair,axis=0),axis=0,repeats=negative_ratio),newshape=(-1,2))
    np.random.shuffle(train_set); train_set = train_set[:batch_size]
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,[train_set.shape[0],3])],axis = -1)
    return train_set

def get_train_set1(batch_size = batch_size):
    train_set = train_pair
    np.random.shuffle(train_set);
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,train_set.shape)],axis = -1)
    return train_set

def get_trgat(node_size,rel_size,node_hidden,rel_hidden,triple_size,n_attn_heads = 2,dropout_rate = 0,gamma = 3,lr = 0.005,depth = 2):
    adj_input = Input(shape=(None,2))
    index_input = Input(shape=(None,2),dtype='int64')
    val_input = Input(shape = (None,))
    rel_adj = Input(shape=(None,2))
    ent_adj = Input(shape=(None,2))
    radj = Input(shape=(None,3),dtype='int64')
    alignment_input = Input(shape=(None,5),dtype = "int32") 
    L = Input(shape=(None,1),dtype = "int32") 
    R = Input(shape=(None,1),dtype = "int32") 

    ent_emb = TokenEmbedding(node_size,node_hidden,trainable = True)(val_input) 
    rel_emb = TokenEmbedding(rel_size,node_hidden,trainable = True)(val_input)
    

    E = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    R = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)

    # avg = AvgClass(node_size,rel_size)
    # ent_feature,rel_feature = avg([ent_emb,rel_emb,radj])
       
    def avg(tensor,size,highway = None):
        adj = K.cast(K.squeeze(tensor[0],axis = 0),dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(adj[:,0],dtype = 'float32'), dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        if highway != None:
            #l_adj = highway([l_adj,tensor[1]])
            l_adj = 0.5 * l_adj + 0.5 * tensor[1]
        return l_adj

    rel_feature = Lambda(avg,arguments={'size':rel_size})([rel_adj,rel_emb])
    ent_feature = Lambda(avg,arguments={'size':node_size})([ent_adj,ent_emb])


    encoder1 = NR_GraphAttention(node_size,activation="relu",
                                       rel_size = rel_size,
                                       depth = 1,
                                       attn_heads=n_attn_heads,
                                       triple_size = triple_size,
                                       attn_heads_reduction='average',   
                                       dropout_rate=dropout_rate)
  
    encoder2 = NR_GraphAttention(node_size,activation="relu",
                                    rel_size = rel_size,
                                    depth = 1,
                                    attn_heads=n_attn_heads,
                                    triple_size = triple_size,
                                    attn_heads_reduction='average',   
                                    dropout_rate=dropout_rate)
    
    sim_layer = sim_mat()
    loss_layer = my_loss(batch_size,dropout_rate)
        
    opt1 = [rel_emb,adj_input,index_input,val_input,radj]
    weight =  tf.Variable(1 * [-0.5] + (num_el-1) * [1.0], trainable=False)
    weight = 5 * tf.math.softmax(weight)
    loss = 0
    sim = 0

    f = weight[0] * ent_feature 
    loss += loss_layer([f,alignment_input])
    sim_mat = sim_mat([f,L,R])
    
    middle_F = encoder1([ent_feature] + opt1)
    f = weight[1] * middle_F 
    loss += loss_layer([f,alignment_input])
    sim_mat += sim_mat([f,L,R])  

    middle_F = encoder2([middle_F] + opt1)
    f = weight[2] * middle_F 
    loss += loss_layer([f,alignment_input])
    sim_mat += sim_mat([f,L,R]) 

    f = weight[3] * rel_feature 
    loss += loss_layer([f,alignment_input])
    sim_mat += sim_mat([f,L,R])
    
    middle_F = encoder1([rel_feature] + opt1)
    f = weight[4] * middle_F 
    loss += loss_layer([f,alignment_input])
    sim_mat += sim_mat([f,L,R])  

    middle_F = encoder2([middle_F] + opt1)
    f = weight[5] * middle_F 
    loss += loss_layer([f,alignment_input])
    sim_mat += sim_mat([f,L,R]) 
   
    inputs = [adj_input,index_input,val_input,rel_adj,ent_adj,radj]
    train_model = keras.Model(inputs = inputs + [alignment_input],outputs = loss)   
    train_model.compile(loss=lambda y_true,y_pred: y_pred,optimizer=tf.keras.optimizers.RMSprop(lr=lr))
    
    feature_model = keras.Model(inputs = inputs+ [L,R],outputs = sim_mat)
    return train_model,feature_model,losses_model


In [32]:
model,get_emb,get_losses = get_trgat(dropout_rate=0.30,node_size=node_size,rel_size=rel_size,n_attn_heads = 1,depth=2,gamma =3,node_hidden=100,rel_hidden = 100,triple_size = triple_size)
model.summary(); initial_weights = model.get_weights()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 token_embedding (TokenEmbeddin  (38960, 100)        3896000     ['input_3[0][0]']                
 g)                                                                                               
                                                                                                  
 input_4 (InputLayer)           [(None, None, 2)]    0           []                               
                                                                                                  
 token_embedding_1 (TokenEmbedd  (6048, 100)         604800      ['input_3[0][0]']            

In [33]:
train_pair = train_pair_main
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True  
sess = tf.compat.v1.Session(config=config) 

rest_set_1 = [e1 for e1, e2 in dev_pair]
rest_set_2 = [e2 for e1, e2 in dev_pair]
np.random.shuffle(rest_set_1)
np.random.shuffle(rest_set_2)

epoch = 1200
for turn in range(5):
    print("iteration %d start."%turn)
    for i in trange(epoch):
        train_set = get_train_set()
        inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,train_set]
        inputs = [np.expand_dims(item,axis=0) for item in inputs]
        model.train_on_batch(inputs,np.zeros((1,1)))
        if i%100 == 99:
            print(get_losses_values())
            CSLS_test()
           
    new_pair = []
    
    sim_mat = get_sim_mat()
    A,_ = eval_alignment_by_mat(sim_mat, [1, 5, 10], 16,10,True,False)
    B,_ = eval_alignment_by_mat(np.transpose(sim_mat),[1, 5, 10], 16,10,True,False)
    A = sorted(list(A)); B = sorted(list(B))
    for a,b in A:
        if  B[b][1] == a:
            new_pair.append([rest_set_1[a],rest_set_2[b]])
    print("generate new semi-pairs: %d." % len(new_pair))
    train_pair = np.concatenate([train_pair,np.array(new_pair)],axis = 0)
    for e1,e2 in new_pair:
        if e1 in rest_set_1:
            rest_set_1.remove(e1) 
        
    for e1,e2 in new_pair:
        if e2 in rest_set_2:
            rest_set_2.remove(e2) 

iteration 0 start.


  8%|▊         | 99/1200 [02:23<26:00,  1.42s/it]

[0.00271726 0.00462225 0.00484065 0.00401469 0.00400555 0.00368681
 0.00336626]


  8%|▊         | 100/1200 [02:47<2:26:22,  7.98s/it]

accurate results: hits@[1, 5, 10] = [65.37 87.05 91.98], mr = 12.651, mrr = 0.750, time = 18.745 s 


...

[0.00393096 0.00635416 0.00677315 0.00548608 0.00573814 0.0052691
 0.00476739]


 17%|█▋        | 200/1200 [05:25<2:04:57,  7.50s/it]

accurate results: hits@[1, 5, 10] = [68.89 88.7  92.72], mr = 12.160, mrr = 0.776, time = 18.925 s 


 22%|██▏       | 259/1200 [06:48<24:43,  1.58s/it]


KeyboardInterrupt: ignored